In [3]:
import FinMind
import pandas as pd 
import numpy as np
import talib
import math
import matplotlib.pyplot as plt
import matplotlib

from FinMind.data import DataLoader
from sklearn.decomposition import PCA

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [42]:
# 2330 2454 2303 2379
stock_id = '2379'

In [43]:
api = DataLoader()
df = api.taiwan_stock_daily(
    stock_id=stock_id,
    start_date='2012-12-01',
    end_date='2021-12-31'
)
df.drop(columns=['stock_id', 'Trading_money', 'spread', 'Trading_turnover'],inplace=True)
df.rename(columns={'Trading_Volume':'volume', 'max':'high', 'min':'low'},inplace=True)
df

,date,volume,open,high,low,close
0,2012-12-03,2163791,58.7,59.8,58.6,58.8
1,2012-12-04,2262898,58.8,59.7,58.3,59.7
2,2012-12-05,2270207,59.9,60.0,59.2,59.4
3,2012-12-06,5228980,59.8,61.0,59.3,59.3
4,2012-12-07,2593751,59.8,60.0,58.7,58.8
...,...,...,...,...,...,...
2222,2021-12-24,932808,575.0,578.0,563.0,564.0
2223,2021-12-27,985453,569.0,579.0,567.0,576.0
2224,2021-12-28,916325,578.0,585.0,577.0,583.0
2225,2021-12-29,630837,582.0,584.0,576.0,578.0


In [44]:
close_array = np.array(df['close'])

In [45]:
label = []
for i in range(0, len(close_array)):
    length = len(close_array[i:])
    if length > 21:
        length = 21
    if length == 21:
        for j in range(0, length):
            upper_bound = close_array[i] * 1.04
            lower_bound = close_array[i] * 0.98
            if close_array[i+j] >= upper_bound:
                label.append(1)
                break
            elif close_array[i+j] <= lower_bound:
                label.append(2)
                break
        if len(label) != i+1:
            label.append(0)
    else:
        for j in range(0, length):
            upper_bound = close_array[i] * 1.04
            lower_bound = close_array[i] * 0.98
            if close_array[i+j] >= upper_bound:
                label.append(1)
                break
            elif close_array[i+j] <= lower_bound:
                label.append(2)
                break
        if len(label) != i+1:
            label.append(np.nan)

In [46]:
df['label'] = label
df

,date,volume,open,high,low,close,label
0,2012-12-03,2163791,58.7,59.8,58.6,58.8,1.0
1,2012-12-04,2262898,58.8,59.7,58.3,59.7,1.0
2,2012-12-05,2270207,59.9,60.0,59.2,59.4,1.0
3,2012-12-06,5228980,59.8,61.0,59.3,59.3,1.0
4,2012-12-07,2593751,59.8,60.0,58.7,58.8,1.0
...,...,...,...,...,...,...,...
2222,2021-12-24,932808,575.0,578.0,563.0,564.0,NaN
2223,2021-12-27,985453,569.0,579.0,567.0,576.0,NaN
2224,2021-12-28,916325,578.0,585.0,577.0,583.0,NaN
2225,2021-12-29,630837,582.0,584.0,576.0,578.0,NaN


In [47]:
df['MA-5'] = (df['close'] - talib.MA(df.close, timeperiod=5, matype=0)) / talib.MA(df.close, timeperiod=5, matype=0)
df['MA-10'] = (df['close'] - talib.MA(df.close, timeperiod=10, matype=0)) / talib.MA(df.close, timeperiod=10, matype=0)
df['MA-20'] = (df['close'] - talib.MA(df.close, timeperiod=20, matype=0)) / talib.MA(df.close, timeperiod=20, matype=0)
df['MA-60'] = (df['close'] - talib.MA(df.close, timeperiod=60, matype=0)) / talib.MA(df.close, timeperiod=60, matype=0)
df['RSI-14'] = talib.RSI(df.close, timeperiod=14)
df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(df.close)
# df.dropna(inplace=True)

In [48]:
df

,date,volume,open,high,low,close,label,MA-5,MA-10,MA-20,MA-60,RSI-14,MACD,MACD_signal,MACD_hist
0,2012-12-03,2163791,58.7,59.8,58.6,58.8,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-12-04,2262898,58.8,59.7,58.3,59.7,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-12-05,2270207,59.9,60.0,59.2,59.4,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012-12-06,5228980,59.8,61.0,59.3,59.3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012-12-07,2593751,59.8,60.0,58.7,58.8,1.0,-0.006757,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222,2021-12-24,932808,575.0,578.0,563.0,564.0,NaN,-0.003886,0.003380,0.008133,0.083903,55.867462,9.324458,10.052382,-0.727924
2223,2021-12-27,985453,569.0,579.0,567.0,576.0,NaN,0.009110,0.021096,0.025915,0.103448,60.779913,9.690254,9.979956,-0.289702
2224,2021-12-28,916325,578.0,585.0,577.0,583.0,NaN,0.016388,0.026589,0.035800,0.112666,63.343189,10.424820,10.068929,0.355891
2225,2021-12-29,630837,582.0,584.0,576.0,578.0,NaN,0.005917,0.013324,0.024278,0.099312,60.311108,10.482674,10.151678,0.330996


# CONCAT

In [49]:
api = DataLoader()
df2 = api.taiwan_stock_margin_purchase_short_sale(
    stock_id="2330",
    start_date='2012-12-01',
    end_date='2021-12-31'
)
df2.drop(columns=['stock_id'],inplace=True)
df2

,date,MarginPurchaseBuy,MarginPurchaseCashRepayment,MarginPurchaseLimit,MarginPurchaseSell,MarginPurchaseTodayBalance,MarginPurchaseYesterdayBalance,Note,OffsetLoanAndShort,ShortSaleBuy,ShortSaleCashRepayment,ShortSaleLimit,ShortSaleSell,ShortSaleTodayBalance,ShortSaleYesterdayBalance
0,2012-12-03,2961,0,6480690,1241,15254,13534,,953,381,0,6480690,172,8392,8601
1,2012-12-04,3382,2,6480690,2022,16612,15254,,1185,828,1,6480690,216,7779,8392
2,2012-12-05,2392,0,6480690,2352,16652,16612,,4235,347,2,6480690,432,7862,7779
3,2012-12-06,1526,53,6480690,1510,16615,16652,,710,123,2,6480690,132,7869,7862
4,2012-12-07,1560,16,6480690,3080,15079,16615,,1290,61,2,6480690,1051,8857,7869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222,2021-12-24,752,14,6482595,343,24705,24310,,3,3,0,6482595,18,74,59
2223,2021-12-27,886,5,6482595,696,24890,24705,,1,3,0,6482595,13,84,74
2224,2021-12-28,738,4,6482595,1185,24439,24890,,3,2,0,6482595,63,145,84
2225,2021-12-29,1481,25,6482595,843,25052,24439,,2,9,7,6482595,28,157,145


In [50]:
# df_tmp = pd.concat([df.set_index('date'),df2.set_index('date')], axis=1, join='inner').reset_index()
df_tmp = df.merge(df2, how='inner', on='date')
df_tmp

,date,volume,open,high,low,close,label,MA-5,MA-10,MA-20,...,MarginPurchaseTodayBalance,MarginPurchaseYesterdayBalance,Note,OffsetLoanAndShort,ShortSaleBuy,ShortSaleCashRepayment,ShortSaleLimit,ShortSaleSell,ShortSaleTodayBalance,ShortSaleYesterdayBalance
0,2012-12-03,2163791,58.7,59.8,58.6,58.8,1.0,NaN,NaN,NaN,...,15254,13534,,953,381,0,6480690,172,8392,8601
1,2012-12-04,2262898,58.8,59.7,58.3,59.7,1.0,NaN,NaN,NaN,...,16612,15254,,1185,828,1,6480690,216,7779,8392
2,2012-12-05,2270207,59.9,60.0,59.2,59.4,1.0,NaN,NaN,NaN,...,16652,16612,,4235,347,2,6480690,432,7862,7779
3,2012-12-06,5228980,59.8,61.0,59.3,59.3,1.0,NaN,NaN,NaN,...,16615,16652,,710,123,2,6480690,132,7869,7862
4,2012-12-07,2593751,59.8,60.0,58.7,58.8,1.0,-0.006757,NaN,NaN,...,15079,16615,,1290,61,2,6480690,1051,8857,7869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222,2021-12-24,932808,575.0,578.0,563.0,564.0,NaN,-0.003886,0.003380,0.008133,...,24705,24310,,3,3,0,6482595,18,74,59
2223,2021-12-27,985453,569.0,579.0,567.0,576.0,NaN,0.009110,0.021096,0.025915,...,24890,24705,,1,3,0,6482595,13,84,74
2224,2021-12-28,916325,578.0,585.0,577.0,583.0,NaN,0.016388,0.026589,0.035800,...,24439,24890,,3,2,0,6482595,63,145,84
2225,2021-12-29,630837,582.0,584.0,576.0,578.0,NaN,0.005917,0.013324,0.024278,...,25052,24439,,2,9,7,6482595,28,157,145


In [51]:
# name 去掉 依照日期去加總but sell
api = DataLoader()
df3 = api.taiwan_stock_institutional_investors(
    stock_id="2330",
    start_date='2012-12-01',
    end_date='2021-12-31'
)
df3.drop(columns=['stock_id', 'name'],inplace=True)
df3

,date,buy,sell
0,2012-12-03,29226286,33878279
1,2012-12-03,300000,118000
2,2012-12-03,1922000,600000
3,2012-12-04,18404076,35004982
4,2012-12-04,60000,269000
...,...,...,...
9414,2021-12-30,0,0
9415,2021-12-30,276000,381177
9416,2021-12-30,552000,227000
9417,2021-12-30,117157,83000


In [52]:
df3 = df3.groupby('date').sum()
df3.reset_index(inplace=True)
df3

,date,buy,sell
0,2012-12-03,31448286,34596279
1,2012-12-04,21721076,35601982
2,2012-12-05,38537440,42388475
3,2012-12-06,27568089,25196068
4,2012-12-07,24588000,17703205
...,...,...,...
2225,2021-12-24,8318626,3297439
2226,2021-12-27,11587578,5092742
2227,2021-12-28,27733167,7384655
2228,2021-12-29,16187458,10118518


In [53]:
# df_tmp = pd.concat([df_tmp.set_index('date'),df3.set_index('date')], axis=1, join='inner').reset_index()
df_tmp = df_tmp.merge(df3, how='inner', on='date')
df_tmp

,date,volume,open,high,low,close,label,MA-5,MA-10,MA-20,...,Note,OffsetLoanAndShort,ShortSaleBuy,ShortSaleCashRepayment,ShortSaleLimit,ShortSaleSell,ShortSaleTodayBalance,ShortSaleYesterdayBalance,buy,sell
0,2012-12-03,2163791,58.7,59.8,58.6,58.8,1.0,NaN,NaN,NaN,...,,953,381,0,6480690,172,8392,8601,31448286,34596279
1,2012-12-04,2262898,58.8,59.7,58.3,59.7,1.0,NaN,NaN,NaN,...,,1185,828,1,6480690,216,7779,8392,21721076,35601982
2,2012-12-05,2270207,59.9,60.0,59.2,59.4,1.0,NaN,NaN,NaN,...,,4235,347,2,6480690,432,7862,7779,38537440,42388475
3,2012-12-06,5228980,59.8,61.0,59.3,59.3,1.0,NaN,NaN,NaN,...,,710,123,2,6480690,132,7869,7862,27568089,25196068
4,2012-12-07,2593751,59.8,60.0,58.7,58.8,1.0,-0.006757,NaN,NaN,...,,1290,61,2,6480690,1051,8857,7869,24588000,17703205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222,2021-12-24,932808,575.0,578.0,563.0,564.0,NaN,-0.003886,0.003380,0.008133,...,,3,3,0,6482595,18,74,59,8318626,3297439
2223,2021-12-27,985453,569.0,579.0,567.0,576.0,NaN,0.009110,0.021096,0.025915,...,,1,3,0,6482595,13,84,74,11587578,5092742
2224,2021-12-28,916325,578.0,585.0,577.0,583.0,NaN,0.016388,0.026589,0.035800,...,,3,2,0,6482595,63,145,84,27733167,7384655
2225,2021-12-29,630837,582.0,584.0,576.0,578.0,NaN,0.005917,0.013324,0.024278,...,,2,9,7,6482595,28,157,145,16187458,10118518


In [54]:
api = DataLoader()
df4 = api.taiwan_stock_shareholding(
    stock_id="2330",
    start_date='2012-12-01',
    end_date='2021-12-31'
)
df4.drop(columns=['stock_id', 'note', 'stock_name'],inplace=True)
df4

,date,InternationalCode,ForeignInvestmentRemainingShares,ForeignInvestmentShares,ForeignInvestmentRemainRatio,ForeignInvestmentSharesRatio,ForeignInvestmentUpperLimitRatio,ChineseInvestmentUpperLimitRatio,NumberOfSharesIssued,RecentlyDeclareDate
0,2012-12-03,TW0002330008,5781930874,20138778485,22.30,77.69,100.0,100.0,25920709359,2012-05-18
1,2012-12-04,TW0002330008,5797412780,20123296579,22.36,77.63,100.0,100.0,25920709359,2012-05-18
2,2012-12-05,TW0002330008,5801543815,20119165544,22.38,77.61,100.0,100.0,25920709359,2012-05-18
3,2012-12-06,TW0002330008,5800491794,20120217565,22.37,77.62,100.0,100.0,25920709359,2012-05-18
4,2012-12-07,TW0002330008,5792277999,20128431360,22.34,77.65,100.0,100.0,25920709359,2012-05-18
...,...,...,...,...,...,...,...,...,...,...
2240,2021-12-24,TW0002330008,6497484536,19432895922,25.05,74.94,100.0,100.0,25930380458,2021-05-19
2241,2021-12-27,TW0002330008,6490938044,19439442414,25.03,74.96,100.0,100.0,25930380458,2021-05-19
2242,2021-12-28,TW0002330008,6471358071,19459022387,24.95,75.04,100.0,100.0,25930380458,2021-05-19
2243,2021-12-29,TW0002330008,6465968642,19464411816,24.93,75.06,100.0,100.0,25930380458,2021-05-19


In [55]:
# df_tmp = pd.concat([df_tmp.set_index('date'),df4.set_index('date')], axis=1, join='inner').reset_index()
df_tmp = df_tmp.merge(df4, how='inner', on='date')
df_tmp

,date,volume,open,high,low,close,label,MA-5,MA-10,MA-20,...,sell,InternationalCode,ForeignInvestmentRemainingShares,ForeignInvestmentShares,ForeignInvestmentRemainRatio,ForeignInvestmentSharesRatio,ForeignInvestmentUpperLimitRatio,ChineseInvestmentUpperLimitRatio,NumberOfSharesIssued,RecentlyDeclareDate
0,2012-12-03,2163791,58.7,59.8,58.6,58.8,1.0,NaN,NaN,NaN,...,34596279,TW0002330008,5781930874,20138778485,22.30,77.69,100.0,100.0,25920709359,2012-05-18
1,2012-12-04,2262898,58.8,59.7,58.3,59.7,1.0,NaN,NaN,NaN,...,35601982,TW0002330008,5797412780,20123296579,22.36,77.63,100.0,100.0,25920709359,2012-05-18
2,2012-12-05,2270207,59.9,60.0,59.2,59.4,1.0,NaN,NaN,NaN,...,42388475,TW0002330008,5801543815,20119165544,22.38,77.61,100.0,100.0,25920709359,2012-05-18
3,2012-12-06,5228980,59.8,61.0,59.3,59.3,1.0,NaN,NaN,NaN,...,25196068,TW0002330008,5800491794,20120217565,22.37,77.62,100.0,100.0,25920709359,2012-05-18
4,2012-12-07,2593751,59.8,60.0,58.7,58.8,1.0,-0.006757,NaN,NaN,...,17703205,TW0002330008,5792277999,20128431360,22.34,77.65,100.0,100.0,25920709359,2012-05-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222,2021-12-24,932808,575.0,578.0,563.0,564.0,NaN,-0.003886,0.003380,0.008133,...,3297439,TW0002330008,6497484536,19432895922,25.05,74.94,100.0,100.0,25930380458,2021-05-19
2223,2021-12-27,985453,569.0,579.0,567.0,576.0,NaN,0.009110,0.021096,0.025915,...,5092742,TW0002330008,6490938044,19439442414,25.03,74.96,100.0,100.0,25930380458,2021-05-19
2224,2021-12-28,916325,578.0,585.0,577.0,583.0,NaN,0.016388,0.026589,0.035800,...,7384655,TW0002330008,6471358071,19459022387,24.95,75.04,100.0,100.0,25930380458,2021-05-19
2225,2021-12-29,630837,582.0,584.0,576.0,578.0,NaN,0.005917,0.013324,0.024278,...,10118518,TW0002330008,6465968642,19464411816,24.93,75.06,100.0,100.0,25930380458,2021-05-19


In [56]:
# # 股東持股分級表 TaiwanStockHoldingSharesPer
# api = DataLoader()
# df5 = api.taiwan_stock_holding_shares_per(
#     stock_id="2330",
#     start_date='2012-12-01',
#     end_date='2018-12-02'
# )
# # df4.drop(columns=['stock_id', 'date'],inplace=True)
# df5

In [57]:
# # 借券成交明細 TaiwanStockSecuritiesLending
# api = DataLoader()
# df6 = api.taiwan_stock_securities_lending(
#     stock_id="2330",
#     start_date='2012-12-01',
#     end_date='2018-12-02'
# )
# # df4.drop(columns=['stock_id', 'date'],inplace=True)
# df6

In [58]:
# 融券借券賣出 TaiwanDailyShortSaleBalances
api = DataLoader()
df7 = api.taiwan_daily_short_sale_balances(
    stock_id="2330",
    start_date='2012-12-01',
    end_date='2021-12-31'
)
df7.drop(columns=['stock_id'],inplace=True)
df7

,MarginShortSalesPreviousDayBalance,MarginShortSalesShortSales,MarginShortSalesShortCovering,MarginShortSalesStockRedemption,MarginShortSalesCurrentDayBalance,MarginShortSalesQuota,SBLShortSalesPreviousDayBalance,SBLShortSalesShortSales,SBLShortSalesReturns,SBLShortSalesAdjustments,SBLShortSalesCurrentDayBalance,SBLShortSalesQuota,SBLShortSalesShortCovering,date
0,8601000,172000,381000,0,8392000,-2109244048,36615444,782000,0,320000,37717444,7573226,0,2012-12-03
1,8392000,216000,828000,1000,7779000,-2109244048,37717444,856000,0,500000,39073444,7752443,0,2012-12-04
2,7779000,432000,347000,2000,7862000,-2109244048,39073444,4613000,702000,2500000,45484444,7954673,0,2012-12-05
3,7862000,132000,123000,2000,7869000,-2109244048,45484444,1638000,27000,600000,47695444,7915223,0,2012-12-06
4,7869000,1051000,61000,2000,8857000,-2109244048,47695444,713000,185000,200000,48423444,7707333,0,2012-12-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2207,59000,18000,3000,0,74000,-2107339478,32603856,1000,24000,0,32580856,7172088,0,2021-12-24
2208,74000,13000,3000,0,84000,-2107339478,32580856,20000,0,0,32600856,7168398,0,2021-12-27
2209,84000,63000,2000,0,145000,-2107339478,32600856,700000,0,0,33300856,7302198,0,2021-12-28
2210,145000,28000,9000,7000,157000,-2107339478,33300856,0,97000,0,33203856,7339945,0,2021-12-29


In [59]:
# df_tmp = pd.concat([df_tmp.set_index('date'),df7.set_index('date')], axis=1, join='inner').reset_index()
df_tmp = df_tmp.merge(df7, how='left', on='date')
df_tmp

,date,volume,open,high,low,close,label,MA-5,MA-10,MA-20,...,MarginShortSalesStockRedemption,MarginShortSalesCurrentDayBalance,MarginShortSalesQuota,SBLShortSalesPreviousDayBalance,SBLShortSalesShortSales,SBLShortSalesReturns,SBLShortSalesAdjustments,SBLShortSalesCurrentDayBalance,SBLShortSalesQuota,SBLShortSalesShortCovering
0,2012-12-03,2163791,58.7,59.8,58.6,58.8,1.0,NaN,NaN,NaN,...,0.0,8392000.0,-2.109244e+09,36615444.0,782000.0,0.0,320000.0,37717444.0,7573226.0,0.0
1,2012-12-04,2262898,58.8,59.7,58.3,59.7,1.0,NaN,NaN,NaN,...,1000.0,7779000.0,-2.109244e+09,37717444.0,856000.0,0.0,500000.0,39073444.0,7752443.0,0.0
2,2012-12-05,2270207,59.9,60.0,59.2,59.4,1.0,NaN,NaN,NaN,...,2000.0,7862000.0,-2.109244e+09,39073444.0,4613000.0,702000.0,2500000.0,45484444.0,7954673.0,0.0
3,2012-12-06,5228980,59.8,61.0,59.3,59.3,1.0,NaN,NaN,NaN,...,2000.0,7869000.0,-2.109244e+09,45484444.0,1638000.0,27000.0,600000.0,47695444.0,7915223.0,0.0
4,2012-12-07,2593751,59.8,60.0,58.7,58.8,1.0,-0.006757,NaN,NaN,...,2000.0,8857000.0,-2.109244e+09,47695444.0,713000.0,185000.0,200000.0,48423444.0,7707333.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222,2021-12-24,932808,575.0,578.0,563.0,564.0,NaN,-0.003886,0.003380,0.008133,...,0.0,74000.0,-2.107339e+09,32603856.0,1000.0,24000.0,0.0,32580856.0,7172088.0,0.0
2223,2021-12-27,985453,569.0,579.0,567.0,576.0,NaN,0.009110,0.021096,0.025915,...,0.0,84000.0,-2.107339e+09,32580856.0,20000.0,0.0,0.0,32600856.0,7168398.0,0.0
2224,2021-12-28,916325,578.0,585.0,577.0,583.0,NaN,0.016388,0.026589,0.035800,...,0.0,145000.0,-2.107339e+09,32600856.0,700000.0,0.0,0.0,33300856.0,7302198.0,0.0
2225,2021-12-29,630837,582.0,584.0,576.0,578.0,NaN,0.005917,0.013324,0.024278,...,7000.0,157000.0,-2.107339e+09,33300856.0,0.0,97000.0,0.0,33203856.0,7339945.0,0.0


In [60]:
df_tmp.to_csv(stock_id + '.csv')